# Lunar Diurnal test case



## Setup and Imports

In [1]:
import sys
import os
from pathlib import Path

# Add parent directory to path for imports
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta

# Import thermal model components
from core.config_manager import ConfigManager, create_default_base_config
from core.thermal_simulation import ThermalSimulator
from modelmain import Simulator
from config import SimulationConfig
from modelmain import fit_blackbody_wn_banded, max_btemp_blackbody, emissionT

from radiance_processor import calculate_radiances_from_results, recompute_radiance_with_angles


# Set up plotting style
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['lines.linewidth'] = 2

print("Imports completed successfully!")

Imports completed successfully!


## Create and Load Base Simulation Configuration

Create and load in a default config file. 

In [2]:
# Option 1: Create a new default configuration
config_path = "../configs/base_configs/analysis_template.yaml"

# Create config directory if it doesn't exist
os.makedirs(os.path.dirname(config_path), exist_ok=True)

# Create default configuration for analysis
create_default_base_config(config_path)

# Load the configuration
config_manager = ConfigManager(config_path)

# Create base configuration
base_config = config_manager.create_config()

print(f"Configuration created at: {config_path}")
print(f"Loaded configuration.")


Default configuration saved to: ../configs/base_configs/analysis_template.yaml
Configuration created at: ../configs/base_configs/analysis_template.yaml
Loaded configuration.


## Run Isothermal Spectrum Simulation



In [ ]:
# Modify base configuration for our analysis
settings = {
    'diurnal': True,                       # Steady state simulation
    'sun': True,                           # Include solar heating
    'T_fixed': False,                        # Temperature is fixed, prevents thermal evolution
    'enable_diurnal_convergence': False,
    'thermal_evolution_mode': 'two_wave',   #Run thermal evolution later with broadband vis (turned off) and broadband thermal. 
    'RTE_solver': 'disort',
    'output_radiance_mode': 'two_wave',       #Compute spectral radiance in thermal only. 
    'depth_dependent_properties': True,
    'temperature_dependent_properties': True,
    'temp_dependent_cp': True,
    'temp_dependent_k': False,      #Only use for non-RTE model. Temperature-dependence of conductivity comes naturally in RTE model. 
    'temp_change_threshold': 1.0,
    'P': 29.5306*24*60*60,
    'dtfac': 50000,
    'minsteps': 100000,
    'ndays': 2,
    'observer_mu': 1.0,                     # Observer zenith angle (0 for nadir)
    'Et': 10000.0,                          # Mean extinction coefficient. For phi=0.37, 50300. For phi=0.60, 81600. 
    'eta': 1.0,                             #Vis/thermal extinction coefficient ratio. 
    'ssalb_therm': 0.1,                     # Single scattering albedo for thermal radiation (average from mie code)
    'g_therm': 0.0,                         # Asymmetry parameter for thermal radiation (average from mie code)
    'ssalb_vis': 0.50,
    'g_vis': 0.0,
    'R_base': 0.0,                          # Global reflectivity value for substrate     
    'disort_space_temp': 0.0,              # Cold shroud temperature
    'single_layer': True,                   # Use single-layer model
    'dust_thickness': 5.0,                 # 10 cm
    'T_bottom': 260,                        # Sample base fixed at 500 K
    'bottom_bc': 'dirichlet',               # Bottom boundary condition 
    'crater': False,
    'nstr_out': 8,                # Number of streams for output
    'nmom_out': 8,                # Number of moments for output
    'nstr': 4,
    'nmom': 4
}

# Create configuration with overrides
config1 = config_manager.create_config(settings)

print("Running baseline simulation...")
print(f"Configuration: Et={config1.Et}, k_dust={config1.k_dust}, thickness={config1.dust_thickness}m")

# Run simulation
sim1 = Simulator(config1)
T_out1, phi_vis1, phi_therm1, T_surf1, t_out1 = sim1.run()


print(f"Simulation completed! Output shape: {T_out1.shape}")
print(f"Time range: {t_out1[0]:.0f} to {t_out1[-1]:.0f} seconds")
print(f"Surface temperature range: {T_surf1.min():.1f} to {T_surf1.max():.1f} K")

Running baseline simulation...
Configuration: Et=10000.0, k_dust=0.00055, thickness=5.0m
Using depth-dependent properties:
  Surface density: 1100.0 kg/m³
  Deep density: 1800.0 kg/m³
  Surface conductivity: 7.40e-04 W/m/K
  Deep conductivity: 3.40e-03 W/m/K
Time step: 3.065335 s, Steps per day: 832354
Initial temperature-dependent heat capacity calculated (T=260.0K)
  cp range: 693.1 - 693.1 J/kg/K
Temperature-dependent property tracking initialized
Thermal evolution mode: two_wave
Time step 0/1664708


## Plot hapke diurnal model results

In [ ]:
from matplotlib.ticker import FuncFormatter, LogLocator

mu = 1.0
btemp = emissionT(T_out1[1:-1,:], sim1.grid.x_boundaries,0.0,mu)
plt.plot(t_out1, btemp, label='Hapke BTemp', color='blue')
plt.xlabel('Time (s)')
plt.ylabel('Brightness Temperature (K)')
plt.legend()
plt.show()

# Plot temperature profiles vs depth (log scale) with a second x-axis for tau (opacity, non-log)
fig, ax1 = plt.subplots()
time_fracs = np.linspace(0, 1, len(sim1.t_out))
plot_fracs = np.linspace(0, 0.9, 10)
for frac in plot_fracs:
    idx = np.argmin(np.abs(time_fracs - frac))
    if hasattr(sim1.cfg.Et,'shape'):
        x_m = sim1.grid.x[1:] / sim1.cfg.Et[1:]
    else:
        x_m = sim1.grid.x[1:] / sim1.cfg.Et
    ax1.semilogx(x_m, T_out1[1:, idx], label=f't+{frac:.1f}P')

ax1.set_xlabel('Depth')
ax1.set_ylabel('Temperature (K)')
ax1.legend()

# Set custom ticks for more human-readable units
def depth_formatter(x, pos):
    if x < 1e-3:
        return f"{x*1e6:.0f} µm"
    elif x < 1:
        return f"{x*1e3:.2f} mm"
    else:
        return f"{x:.2f} m"

ax1.set_xscale('log')
ax1.xaxis.set_major_locator(LogLocator(base=10))
ax1.xaxis.set_major_formatter(FuncFormatter(depth_formatter))

# Add a second x-axis for tau (opacity)
def tau_formatter(tau, pos):
    return f"{tau:.2f}"

# The tau values (opacity) are just sim1.grid.x[1:]
tau_vals = sim1.grid.x[1:]

# Create the secondary axis
ax2 = ax1.twiny()
ax2.set_xlim(ax1.get_xlim())
# Convert the log-scale x_m limits back to tau
xlim = ax1.get_xlim()
if hasattr(sim1.cfg.Et,'shape'):
    tau_min = xlim[0] * sim1.cfg.Et[1]
    tau_max = xlim[1] * sim1.cfg.Et[-1]
else:
    tau_min = xlim[0] * sim1.cfg.Et
    tau_max = xlim[1] * sim1.cfg.Et
ax2.set_xscale('log')
ax2.set_xlim(tau_min, tau_max)
ax2.xaxis.set_major_locator(LogLocator(base=10))
ax2.xaxis.set_major_formatter(FuncFormatter(tau_formatter))
ax2.set_xlabel('Optical Depth (tau)')

plt.tight_layout()
plt.show()



In [ ]:
settings2 = settings.copy()

settings2['use_RTE'] = False
settings2['albedo'] = 0.1159
settings2['em'] = 0.9503
settings2['temp_dependent_k'] = True
settings2['temp_change_threshold'] = 0.001
settings2['k_dust_auto'] = False
settings2['minsteps'] = 2000000
settings2['flay'] = 0.01

config2 = config_manager.create_config(settings2)

print("Running baseline simulation...")

# Run simulation
sim2 = Simulator(config2)
T_out2, phi_vis2, phi_therm2, T_surf2, t_out2 = sim2.run()

In [ ]:
from matplotlib.ticker import FuncFormatter, LogLocator

mu = 1.0
btemp = emissionT(T_out1[1:-1,:], sim1.grid.x_boundaries,0.0,mu)
plt.plot(t_out1, btemp, label='Hapke BTemp', color='blue')
plt.plot(t_out2,T_surf2,label='Traditional model surface temp', color='red')
plt.plot(t_out2,T_out2[1,:])
plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('Brightness Temperature (K)')
plt.legend()
plt.show()


fig, ax1 = plt.subplots()
time_fracs = np.linspace(0, 1, len(sim2.t_out))
plot_fracs = np.linspace(0, 0.9, 10)
for frac in plot_fracs:
    idx = np.argmin(np.abs(time_fracs - frac))
    x_m = sim2.grid.x[1:] / sim2.cfg.Et[1:]
    ax1.semilogx(x_m, T_out2[1:, idx], label=f't+{frac:.1f}P')

ax1.set_xlabel('Depth')
ax1.set_ylabel('Temperature (K)')
ax1.legend()
